# Предсказать объема трат по категориям

Необходимо предсказать объем трат по каждой из 184 категорий на каждый день следующего месяца. Итоговый файл должен содержать предсказания по 184 * 30 = 5520 объектам. Объем трат в конкретной категории считается как сумма всех расходных транзакций в текущей категории по всем пользователям.

In [3]:
import os
import pandas as pd
import numpy as np

from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression

Читаем входные файлы с данными

In [4]:
path = '~/Python And Data Analysis/data'
transactions = pd.read_csv(os.path.join(path, 'transactions.csv'))

In [6]:
transactions.head()

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id
0,39026145,0 10:23:26,4814,1030,-2245.92,NaN
1,39026145,1 10:19:29,6011,7010,56147.89,NaN
2,39026145,1 10:20:56,4829,2330,-56147.89,NaN
3,39026145,1 10:39:54,5499,1010,-1392.47,NaN
4,39026145,2 15:33:42,5499,1010,-920.83,NaN


Берём расходные транзакции и формируем тестовую выборку

In [7]:
train_transactions = transactions[transactions.amount < 0].copy()
train_transactions['day'] = train_transactions.tr_datetime.apply(lambda dt: dt.split()[0]).astype(int)

In [11]:
test_transactions = pd.DataFrame(columns = train_transactions.mcc_code.unique(), 
                                 index = np.arange(1, 31) + train_transactions.day.max())
test_transactions = test_transactions.unstack().reset_index().dropna(axis=1)
test_transactions.columns = ['mcc_code', 'day']

In [12]:
train_grid = pd.DataFrame(columns = train_transactions.mcc_code.unique(), 
                          index = train_transactions.day.unique())
train_grid = train_grid.unstack().reset_index().dropna(axis=1)
train_grid.columns = ['mcc_code', 'day']

Добавляем признаки, относящиеся к дате.

In [13]:
for tr_table in [train_transactions, test_transactions, train_grid]:
    tr_table['week_num'] = tr_table['day'] // 7
    tr_table['week_day'] = tr_table['day'] % 7
    tr_table['month_num'] = tr_table['day'] // 30
    tr_table['month_day'] = tr_table['day'] % 30

In [14]:
tr_table.head()

,mcc_code,day,week_num,week_day,month_num,month_day
0,4814,0,0,0,0,0
1,4814,1,0,1,0,1
2,4814,2,0,2,0,2
3,4814,3,0,3,0,3
4,4814,4,0,4,0,4


In [15]:
train_transactions = pd.merge(train_grid, 
                             train_transactions.groupby(['day', 'week_num', 'week_day', 'month_num', 
                                                        'month_day', 'mcc_code'])[['amount']]
                             .sum().reset_index(), how='left') \
                             .fillna(0)

Для каждого mcc_code получим данные о сумме транзакий за каждый день. Используем для обучения только временные признаки.

In [16]:
train_transactions.month_num.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [17]:
for day_shift in [-1, 0, 1]:
    for month_shift in train_transactions.month_num.unique()[1:]:
        train_shift = train_transactions.copy()
        train_shift['month_num'] += month_shift
        train_shift['month_day'] += day_shift
        train_shift['amount_day_{}_{}'.format(day_shift, month_shift)] = np.log(-train_shift['amount'] + 1)
        train_shift = train_shift[['month_num', 'month_day', 'mcc_code', 
                                   'amount_day_{}_{}'.format(day_shift, month_shift)]]

        train_transactions = pd.merge(train_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)
        test_transactions = pd.merge(test_transactions, train_shift, 
                                      on=['month_num', 'month_day', 'mcc_code'], how='left').fillna(0)

In [21]:
train_transactions.head()

,mcc_code,day,week_num,week_day,month_num,month_day,amount,amount_day_-1_1,amount_day_-1_2,amount_day_-1_3,...,amount_day_1_6,amount_day_1_7,amount_day_1_8,amount_day_1_9,amount_day_1_10,amount_day_1_11,amount_day_1_12,amount_day_1_13,amount_day_1_14,amount_day_1_15
0,4814,0,0,0,0,0,-11098744.26,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4814,1,0,1,0,1,-7881825.53,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4814,2,0,2,0,2,-6777480.45,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4814,3,0,3,0,3,-9277943.73,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4814,4,0,4,0,4,-9999757.21,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Обучаемся на всех данных

In [22]:
shift = 500
train = pd.get_dummies(train_transactions, columns=['mcc_code'])
test = pd.get_dummies(test_transactions, columns=['mcc_code'])
c = train.columns.difference(['amount'])

clf = LinearRegression()
clf.fit(train[c], np.log(-train['amount'] + shift))

LinearRegression()

Предсказываем объём трат для тестовых данных и создаём файл с ответом

In [25]:
test_transactions['volume'] = np.e ** clf.predict(test[c]) - shift

In [27]:
test_transactions[['mcc_code', 'day', 'volume']]

,mcc_code,day,volume
0,4814,457,1.204552e+07
1,4814,458,1.226663e+07
2,4814,459,1.235530e+07
3,4814,460,1.224792e+07
4,4814,461,1.250476e+07
...,...,...,...
5515,9211,482,4.866006e+02
5516,9211,483,8.830576e+02
5517,9211,484,5.148354e+02
5518,9211,485,4.932042e+02


In [28]:
test_transactions[['mcc_code', 'day', 'volume']].to_csv('spending_mcc_predictions.csv', index=False)